In [2]:
# https://www.guru99.com/gecko-marionette-driver-selenium.html
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.firefox.options import Options
import http.client, urllib.request, urllib.parse, urllib.error, base64
import shutil
import datetime
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import os

def getDriver():
    
    #download_folder = "C:/Users/Ian/Documents/GitHub/delitos_chile/Poder Judicial/Descargas"
    
    # Options se ha importado desde Selenium
    
    options = Options()
    options.set_preference("browser.download.dir", r"C:\Users\Ian\Documents\GitHub\delitos_chile\Poder Judicial\Descargas")
    #options.set_preference("browser.download.folderList", 2)
    options.set_preference("browser.download.manager.showWhenStarting", False)
    options.set_preference("browser.helperApps.neverAsk.saveToDisk", "text/csv") #, "application/json")
    
    #browser = webdriver.WebDriver(firefox_profile=profile)
    #driver = webdriver.WebDriver(firefox_profile=profile)
    driver = webdriver.Firefox(options=options)
    driver.set_page_load_timeout("60")
    driver.get("https://www.pjud.cl/estadisticas/estadisticas-cortes-juzgados")
    return driver


In [ ]:
driver = getDriver()

In [9]:
### OJO ###
#MODIFICAR LAS SIGUIENTES TRES LÍNEAS

anio = "2018"  #### MODIFICAR EL AÑO DENTRO DE LAS COMILLAS
ruta_desc = "C:/Users/Ian/Downloads/" #### INSERTAR LA RUTA DE DESCARGAS DE SU PC
ruta_final = "C:/Users/Ian/Documents/GitHub/delitos_chile/Poder Judicial/Descargas/" #### INSERTAR LA RUTA DONDE DESEA ALMACENAR LOS ARCHIVOS QUE DESCARGARÁ



###NO MODIFICAR NADA DEBAJO DE ESTA LÍNEA


rmes = range(0,12)
diccionario = ["R15","R1","R2","R3","R4","R5","R131","R132","R6","R7","R16","R8","R9","R14","R10","R11","R12"]

#Dataframe de almacenamiento de informes sin datos estadísticos
df = pd.DataFrame()
df = df.assign(Corte = None, Tribunal=None,Competencia=None,Informe=None,Año=None,Mes=None)


element = driver.find_elements_by_tag_name('select')
for i in range(1,18):
    #element = driver.find_elements_by_tag_name('select')


    #Nivel 1: Corte
    corte = element[0]
    valorCorte = corte.find_elements_by_tag_name("option")
    nomCorte = valorCorte[i].text
    print(valorCorte[i].text)    
    select_box = driver.find_element_by_id("court_id")
    s1 = Select(select_box)
    s1.select_by_value(str(i))
    time.sleep(2)

    #Nivel 2: Tribunal
    select_box2 = driver.find_element_by_id("first_instance_court_id")        
    s2 = Select(select_box2)

    #Sacamos el tamaño de la lista de tribunales
    tribunal = select_box2.text
    res_list = (tribunal.rstrip().split('\n'))
    tam_trib = len(res_list)
    element = driver.find_elements_by_tag_name('select')
    tribunal = element[1]
    valorTribunal = tribunal.find_elements_by_tag_name("option")

    for j in range(1,tam_trib):
        s2.select_by_visible_text(''+valorTribunal[j].text+'')
        nomTrib = valorTribunal[j].text
        print("-"+valorTribunal[j].text)
        time.sleep(2)

        element = driver.find_elements_by_tag_name('select')

        #Nivel 3: Competencia (si existe)
        if(len(element)==6):
            select_box3 = driver.find_element_by_id("competence_id")        
            s3 = Select(select_box3)
            element = driver.find_elements_by_tag_name('select')
            compe = element[2]
            valorCompe = compe.find_elements_by_tag_name("option")        
            competencia = select_box3.text
            comp_list = (competencia.rstrip().split('\n'))
            tam_comp = len(comp_list)
            for k in range(1,tam_comp):
                s3.select_by_visible_text(''+valorCompe[k].text+'')
                nomComp = valorCompe[k].text
                print("--"+valorCompe[k].text)
                time.sleep(2)
                #Nivel 4: Informe
                select_box4 = driver.find_element_by_id("report_type_code")        
                sInf = Select(select_box4)
                element = driver.find_elements_by_tag_name('select')
                Inf = element[5]
                valorInf = Inf.find_elements_by_tag_name("option")  
                informe = select_box4.text
                inf_list = (informe.rstrip().split('\n'))
                tam_inf = len(inf_list)         
                for l in range(1,tam_inf):
                    sInf.select_by_visible_text(''+valorInf[l].text+'')
                    nomInf = valorInf[l].text
                    print("---"+valorInf[l].text)
                    #Nivel 5: año
                    select_box5 = driver.find_element_by_id("year")
                    s5 = Select(select_box5)
                    s5.select_by_visible_text(anio)
                    element = driver.find_elements_by_tag_name('select')
                    mes = element[3]
                    valorMes = mes.find_elements_by_tag_name("option")
                    for m in range(0,12):
                        # Nivel 6: mes
                        select_box6 = driver.find_element_by_id("month")
                        s6 = Select(select_box6)
                        s6.select_by_visible_text(''+valorMes[m].text+'')
                        nomMes = valorMes[m].text
                        print("----"+valorMes[m].text)
                        btnBuscar = driver.find_element_by_id("search-stats-button")
                        btnBuscar.click()
                        time.sleep(3)
                        driver.execute_script("window.scrollBy(0, 600)")
                        time.sleep(2)
                        flag = True
                        i = 0
                        while flag:
                            try:
                                btnExcel = driver.find_element_by_xpath("/html/body/section/div/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div[1]/button[1]")
                                btnExcel.click()
                                time.sleep(3)
                                driver.execute_script("window.scrollBy(0, -600)")
                                src=ruta_desc+"Poder Judicial - Estadisticas.xlsx"
                                des=ruta_final+diccionario[i]+"_"+nomTrib+"_"+nomComp+"_"+nomInf+"_"+anio+"-"+nomMes+".xlsx"
                                os.replace(src,des)
                                flag = False
                            except:
                                i = i+1
                                print("No existe información de estadísticas")
                                driver.execute_script("window.scrollBy(0, -600)")
                            if (i == 10):
                                flag = False
                                i = 0
                            nueva_fila = { 'Corte':nomCorte, 'Tribunal':nomTrib, 'Competencia':nomComp,'Informe':nomInf,'Año':anio,'Mes':nomMes} # creamos un diccionario
                            df = df.append(nueva_fila, ignore_index=True)
        #Nivel 4: Informe
        else:
            select_box4 = driver.find_element_by_id("report_type_code")
            sInf = Select(select_box4)
            element = driver.find_elements_by_tag_name('select')
            Inf = element[4]
            valorInf = Inf.find_elements_by_tag_name("option")
            informe = select_box4.text
            inf_list = (informe.rstrip().split('\n'))
            tam_inf = len(inf_list)
            for l in range(1,tam_inf):
                sInf.select_by_visible_text(''+valorInf[l].text+'')
                nomInf = valorInf[l].text
                print("--"+valorInf[l].text)
                #Nivel 5: año
                select_box5 = driver.find_element_by_id("year")
                s5 = Select(select_box5)
                s5.select_by_visible_text(anio)    
                element = driver.find_elements_by_tag_name('select')
                mes = element[2]
                valorMes = mes.find_elements_by_tag_name("option")
                for m in range(0,12):
                    # Nivel 6: mes
                    select_box6 = driver.find_element_by_id("month")
                    s6 = Select(select_box6)
                    s6.select_by_visible_text(''+valorMes[m].text+'')
                    time.sleep(2)
                    nomMes = valorMes[m].text
                    print("---"+valorMes[m].text)
                    btnBuscar = driver.find_element_by_id("search-stats-button")
                    btnBuscar.click()
                    time.sleep(3)
                    driver.execute_script("window.scrollBy(0, 600)")
                    time.sleep(2)
                    flag = True
                    i = 0
                    while flag:
                        try:
                            btnExcel = driver.find_element_by_xpath("/html/body/section/div/div[1]/div/div[2]/div/div/div/div/div/div/div[2]/div/div[2]/div/div/div[1]/button[1]")
                            btnExcel.click()
                            time.sleep(3)
                            driver.execute_script("window.scrollBy(0, -600)")
                            src=ruta_desc+"Poder Judicial - Estadisticas.xlsx"
                            des=ruta_final+diccionario[i]+"_"+nomTrib+"_"+nomInf+"_"+anio+"-"+nomMes+".xlsx"                    
                            os.replace(src,des)
                            flag = False
                        except:
                            i = i+1
                            print("No existe información de estadísticas")
                            driver.execute_script("window.scrollBy(0, -600)")
                        if (i == 10):
                            flag = False
                            i = 0
                            nueva_fila = { 'Corte':nomCorte, 'Tribunal':nomTrib, 'Competencia':"",'Informe':nomInf,'Año':anio,'Mes':nomMes} # creamos un diccionario
                            df = df.append(nueva_fila, ignore_index=True)
        print("<-------------------------------------->")
df
df.to_excel("Informes sin datos_"+anio+".xlsx", index = False)

Corte De Apelaciones De Arica
-Corte De Apelaciones De Arica
--Ingresos Corte
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Pendientes Corte
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Términos Corte
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
<-------------------------------------->
-Tribunal De Juicio Oral En Lo Penal De Arica
--Ingresos Causas Por Materia
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Ingresos Causas Por Rol
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Audiencias
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Nov

-4º Juzgado De Letras De Arica
--Ingresos Corte
---Enero
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Febrero
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Marzo
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe i

---Octubre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Noviembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Diciembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe inf

---Septiembre
---Octubre
---Noviembre
---Diciembre
--Ingresos Causas Por Rol
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Audiencias
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Terminos Por Materia
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Terminos Por Rol
---Enero
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Febrero
---Marzo
No existe información de estadísticas
No existe información de estadísticas
No exist

---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
<-------------------------------------->
Corte De Apelaciones De Iquique
-Corte De Apelaciones De Iquique
--Ingresos Corte
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Pendientes Corte
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Términos Corte
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
<-------------------------------------->
-Tribunal De Juicio Oral En Lo Penal De Iquique
--Ingresos Causas Por Materia
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
---Noviembre
---Diciembre
--Ingresos Causas Por Rol
---Enero
---Febrero
---Marzo
---Abríl
---Mayo
---Junio
---Julio
---Agosto
---Septiembre
---Octubre
-

...

----Octubre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
----Noviembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
----Diciembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe 

----Abríl
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
----Mayo
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
----Junio
----Julio
----Agosto
----Septiembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información d

---Mayo
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Junio
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Julio
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de

---Octubre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Noviembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
---Diciembre
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe información de estadísticas
No existe inf

StaleElementReferenceException: Message: The element reference of [object String] "{\"element-6066-11e4-a52e-4f735466cecf\":\"8d3d5916-d739-4b78-9704-64a7fd947693\"}" is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed


In [5]:
print(ruta_final)

C:/Users/Ian/Documents/GitHub/delitos_chile/Poder Judicial/Descargas/


In [6]:
diccionario[i]

'R15'

In [8]:
print(src)

C:/Users/Ian/Downloads/Poder Judicial - Estadisticas.xlsx
